# MLOps Pipeline Coloring Book

This notebook may be used to construct a proto-type pipeline

In [1]:
# Check that package is installed correctly. The KFP SDK version should be >=1.6:
!python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"

KFP SDK version: 1.8.17


TODO: Add docstrings

Terraform (https://github.com/GoogleCloudPlatform/vertex-pipelines-end-to-end-samples/tree/main/terraform):
- Create artifact registry
- Create GS bucket
- Create service account for running pipeline
- IAM Privileges

## Imports

In [2]:
import json
import pandas as pd
from kfp.v2 import compiler, dsl
from kfp.v2.dsl import pipeline, component, Artifact, Dataset, Input, Metrics, Model, Output, InputPath, OutputPath
from google.cloud import aiplatform
from google.cloud import aiplatform_v1
import datetime
from IPython.core.magic import register_cell_magic

from utils import MLOpsBoxer

@register_cell_magic
def execute_and_save(file, cell):
    'Run and save python code block to a file'
    with open(file, 'wt') as fd:
        fd.write(cell)
    code = compile(cell, file, 'exec')
    exec(code, globals())

## Data Loading

In [3]:
@component(
    packages_to_install=[
        "google-cloud-bigquery", 
        "pandas",
        "pyarrow",
        "db_dtypes"
    ],
    base_image="python:3.9",
    output_component_file="create_dataset.yaml"
)
def create_dataset(
    bq_table: str,
    output_data_path: OutputPath("Dataset"),
    project: str
):
    from google.cloud import bigquery
    import pandas as pd
    bq_client = bigquery.Client(project=project)


    def get_query(bq_input_table: str) -> str:
        """Generates BQ Query to read data.

        Args:
        bq_input_table: The full name of the bq input table to be read into
        the dataframe (e.g. <project>.<dataset>.<table>)
        Returns: A BQ query string.
        """
        return f"""
        SELECT *
        FROM `{bq_input_table}`
        """

    def load_bq_data(query: str, client: bigquery.Client) -> pd.DataFrame:
        """Loads data from bq into a Pandas Dataframe for EDA.
        Args:
        query: BQ Query to generate data.
        client: BQ Client used to execute query.
        Returns:
        pd.DataFrame: A dataframe with the requested data.
        """
        df = client.query(query).to_dataframe()
        return df

    dataframe = load_bq_data(get_query(bq_table), bq_client)
    dataframe.to_csv(output_data_path)

## Model Training

In [4]:
@component(
    packages_to_install=[
        "scikit-learn",
        "pandas",
        "joblib",
        "tensorflow"
    ],
    base_image="python:3.9",
    output_component_file="train_model.yaml",
)
def train_model(
    output_model_directory: str,
    dataset: Input[Dataset],
    metrics: Output[Metrics],
    model: Output[Model]
):
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.metrics import roc_curve
    from sklearn.model_selection import train_test_split
    from joblib import dump
    import pandas as pd
    import tensorflow as tf
    import pickle
    import os
    
    def save_model(model, uri):
        """Saves a model to uri."""
        with tf.io.gfile.GFile(uri, 'w') as f:
            pickle.dump(model, f)
    
    df = pd.read_csv(dataset.path)
    labels = df.pop("Class").tolist()
    data = df.values.tolist()
    x_train, x_test, y_train, y_test = train_test_split(data, labels)
    skmodel = DecisionTreeClassifier()
    skmodel.fit(x_train,y_train)
    score = skmodel.score(x_test,y_test)
    print('accuracy is:',score)
    metrics.log_metric("accuracy",(score * 100.0))
    metrics.log_metric("framework", "Scikit Learn")
    metrics.log_metric("dataset_size", len(df))

    output_uri = os.path.join(output_model_directory, f'model.pkl')
    save_model(skmodel, output_uri)
    model.path = output_model_directory

## Uploading & Deploying the Model

In [5]:
@component(
    packages_to_install=[
        "google-cloud-aiplatform"
    ],
    base_image="python:3.9",
    output_component_file="deploy_model.yaml",
)
def deploy_model(
    model: Input[Model],
    project: str,
    region: str,
    vertex_endpoint: Output[Artifact],
    vertex_model: Output[Model]
):
    from google.cloud import aiplatform
    aiplatform.init(project=project, location=region)
    deployed_model = aiplatform.Model.upload(
        display_name="beans-model-pipeline",
        artifact_uri = model.uri,
        serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-24:latest"
    )
    endpoint = deployed_model.deploy(machine_type="n1-standard-4")
    vertex_endpoint.uri = endpoint.resource_name
    vertex_model.uri = deployed_model.resource_name

## Define and Run the Pipeline

In [6]:
%%execute_and_save pipeline.py 
@dsl.pipeline(name='training-pipeline')
def pipeline(bq_table: str,
             output_model_directory: str,
             project: str,
             region: str,
            ):
    
    dataset_task = create_dataset(
        bq_table=bq_table, 
        project=project)

    model_task = train_model(
        output_model_directory=output_model_directory,
        dataset=dataset_task.output)

    deploy_task = deploy_model(
        model=model_task.outputs["model"],
        project=project,
        region=region)

In [7]:
pipeline_params = {
    "bq_table": "sandbox-srastatter.mlops_boxer_test.dry-beans",
    "output_model_directory": f"gs://mlops-boxer-test/trained_models/{datetime.datetime.now()}",
    "project": "sandbox-srastatter",
    "region": "us-central1"
}

In [ ]:
MLOpsBoxer.go(project_id='sandbox-srastatter',
              pipeline_params=pipeline_params)

 BUILDING COMPONENTS 


Listing items under project sandbox-srastatter, location us-central1.

Creating temporary tarball archive of 38 file(s) totalling 97.8 KiB before compression.
Uploading tarball of [..] to [gs://sandbox-srastatter_cloudbuild/source/1671221152.024353-21c61d2f0682413985a4fc149097865a.tgz]


vertex-mlops-af                DOCKER  STANDARD_REPOSITORY               us-central1          Google-managed key  2022-05-04T20:27:05  2022-12-16T19:06:22  575.269
Artifact Registry: vertex-mlops-af already exists in project sandbox-srastatter


Created [https://cloudbuild.googleapis.com/v1/projects/sandbox-srastatter/locations/global/builds/1dbc3dc2-919d-4943-839a-76cf082272a1].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/1dbc3dc2-919d-4943-839a-76cf082272a1?project=1006819402307 ].


gs://sandbox-srastatter-bucket/
GS Bucket: sandbox-srastatter-bucket already exists in project sandbox-srastatter
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "1dbc3dc2-919d-4943-839a-76cf082272a1"

FETCHSOURCE
Fetching storage object: gs://sandbox-srastatter_cloudbuild/source/1671221152.024353-21c61d2f0682413985a4fc149097865a.tgz#1671221152418782
Copying gs://sandbox-srastatter_cloudbuild/source/1671221152.024353-21c61d2f0682413985a4fc149097865a.tgz#1671221152418782...
/ [1 files][ 26.2 KiB/ 26.2 KiB]                                                
Operation completed over 1 objects/26.2 KiB.
BUILD
Starting Step #0 - "Build component: train_model"
Step #0 - "Build component: train_model": Already have image (with digest): gcr.io/cloud-builders/docker
Step #0 - "Build component: train_model": Sending build context to Docker daemon  7.168kB
Step #0 - "Build component: train_model": Step 1/7 : FROM python:3.9
Step #0 - "Build component: 

In [16]:
MLOpsBoxer.generate(project_id='sandbox-srastatter',
                    af_registry_name='mlops-boxer-test',
                    af_registry_location='us-central1',
                    gs_bucket_location='us-central1',
                    gs_bucket_name='mlops-boxer-test',
                    pipeline_params=pipeline_params)

Listing items under project sandbox-srastatter, location us-central1.



mlops-boxer-test               DOCKER  STANDARD_REPOSITORY               us-central1          Google-managed key  2022-12-12T18:44:42  2022-12-13T14:37:22  2992.602
Artifact Registry: mlops-boxer-test already exists in project sandbox-srastatter
gs://mlops-boxer-test/
GS Bucket: mlops-boxer-test already exists in project sandbox-srastatter


### Compile and run the end-to-end ML pipeline
With our full pipeline defined, it's time to compile it:

In [9]:
compiler.Compiler().compile(
    pipeline_func=pipeline, 
    package_path="training_pipeline_job.json"
)

/Users/srastatter/Library/Python/3.9/lib/python/site-packages/kfp/v2/compiler/compiler.py:1290: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  warnings.warn(


Next, kick off a pipeline run:

In [9]:
pipeline_params = {
    "bq_table": "test",
    "output_model_directory": "test",
    "output_data_path": "test",
    "project": "test",
    "region": "test"
}

In [36]:
aiplatform.init(project=PROJECT_ID)
training_job = aiplatform.PipelineJob(
    display_name = "mlops-sp2-init-pipeline-run",
    template_path = "training_pipeline_job.json",
    pipeline_root = PIPELINE_ROOT,
    parameter_values = pipeline_params,
    enable_caching = False
)

training_job.submit(service_account='gia-vertex-ai-custom@poc-gia-2034083092.iam.gserviceaccount.com')
#training_job.run()

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/512807375329/locations/us-central1/pipelineJobs/embeddings-pipeline-20220520195220
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/512807375329/locations/us-central1/pipelineJobs/embeddings-pipeline-20220520195220')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/embeddings-pipeline-20220520195220?project=512807375329
